In [6]:
import pandas as pd
import numpy as np
import warnings
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import keras

#import glob
warnings.filterwarnings('ignore')
#os.chdir("Dataopen_1109")

In [28]:
data1 = pd.read_csv("datasets/dataset_1.csv")
data2 = pd.read_csv("datasets/dataset_2.csv")
data3 = pd.read_csv("datasets/dataset_3.csv")
data4 = pd.read_csv("datasets/dataset_4.csv")
data5 = pd.read_csv("datasets/dataset_5.csv")
data6 = pd.read_csv("datasets/dataset_6.csv")
EUReferendumData = pd.read_csv("datasets/EUReferendumData.csv")
annualPayByConstituency_2019 = pd.read_excel("datasets/annualPayByConstituency_2019.xls")
annualPayByConstituency_2016 = pd.read_excel("datasets/annualPayByConstituency_2016.xls")

EUReferendumData.columns

In [24]:
df_Referendum = EUReferendumData[['Region_Code', 'Area_Code', 'Area', 'Electorate', 'Leave', 'Remain']]
df_Referendum.head()

,Region_Code,Area_Code,Area,Electorate,Leave,Remain
0,E12000006,E06000031,Peterborough,120892,53216,34176
1,E12000006,E06000032,Luton,127612,47773,36708
2,E12000006,E06000033,Southend-on-Sea,128856,54522,39348
3,E12000006,E06000034,Thurrock,109897,57765,22151
4,E12000006,E06000055,Bedford,119530,44569,41497


In [29]:
annualPayByConstituency_2019.columns
df_Pay2019 = annualPayByConstituency_2019[['Description', 'Code', ]]

Index([               'Description',                       'Code',
                       'Unnamed: 2',                     'Median',
         'Annual percentage change',                       'Mean',
       'Annual percentage change.1',                           10,
                                 20,                           25,
                                 30,                           40,
                                 60,                           70,
                                 75,                           80,
                                 90],
      dtype='object')

In [12]:
data5['icb_industry'].unique()

array(['Financial Services', 'Industrial Goods & Services', 'Healthcare',
       'Media', 'Oil & Gas'], dtype=object)

In [26]:
pd.Timestamp(data1.created)

TypeError: Cannot convert input [0    2014-12-10T12:36:33+00:00
1    2017-07-05T00:46:00+00:00
2    2018-05-21T00:42:00+00:00
3    2017-11-02T19:51:00+00:00
4    2014-01-07T22:48:08+00:00
5    2017-12-14T15:26:50+00:00
6    2017-12-23T12:45:00+00:00
7    2019-07-26T07:16:00+00:00
8    2019-04-09T06:13:00+00:00
9    2019-02-07T11:32:00+00:00
Name: created, dtype: object] of type <class 'pandas.core.series.Series'> to Timestamp